In [3]:
# Load
library("tm")
library("SnowballC")
library("wordcloud")
library("RColorBrewer")
library(factoextra)
library(cluster)
library(NbClust)
library(pryr)

#library("lsa")
#library("lda")
mem_used()



Warning message:
"package 'factoextra' was built under R version 3.3.3"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.4.2"

ERROR: Error: package 'ggplot2' could not be loaded


In [ ]:

newsGroups <- c("C:\Users\Dell\Documents\Codes\Machine-Learning\Data Mining - R\20news-18828")

#read in the recursive way
alldir=DirSource(newsGroups, encoding = "UTF-8", recursive=TRUE)



In [ ]:
alldir$filelist


In [ ]:
news <- Corpus(alldir, readerControl=list(reader=readPlain,language="en"))


# data preprocessing
news <- tm_map(news, removeWords,"Subject") 
news <- tm_map(news, removeWords,"Organization") 
news <- tm_map(news, removeWords,"writes") 
news <- tm_map(news, removeWords,"From") 
news <- tm_map(news, removeWords,"lines") 
news <- tm_map(news, removeWords,"NNTP-Posting-Host") 
news <- tm_map(news, removeWords,"article")





In [ ]:


news <- tm_map(news, tolower) ## Convert to Lower Case 

news <- tm_map(news, removeWords, stopwords("english")) ## Remove Stopwords 

news <- tm_map(news, removePunctuation) ## Remove Punctuations 

news <- tm_map(news, stemDocument) ## Stemming 

news <- tm_map(news, removeNumbers) ## Remove Numbers 

news <- tm_map(news, stripWhitespace) ## Eliminate Extra White Spaces 

news <- tm_map(news , PlainTextDocument)

mem_used()



In [ ]:
#DocumentTermMatrix

dtm <- DocumentTermMatrix(news,control=list(wordLengths=c(3,Inf)))
dtm
dim(dtm)
str(dtm)
#inspect(dtm[1:5,1:20])
mem_used()


In [ ]:
#construct an empty vector, to be used for holding class labels of the documents
classvec <- vector()

#loop all the files: for each document, make its parent folder name as the class label of this document, and put the class label value in classvec vector

#alldir$filelist  #display all the files
for (filedir in alldir$filelist) {

classlabel=basename(dirname(filedir))
classvec=c(classvec,classlabel)

}
summary(classvec)



In [ ]:
#factor classvec to let R know it's a categorical variable
classvec <- factor(classvec)

In [ ]:
classvec

In [ ]:

library(Matrix)
library(SparseM)
library(e1071)


In [ ]:

# convert dtm to dtm_matrix using sparse storage
dtm_matrix=as.matrix.csr(as.matrix(dtm))
#specify the features, vector to be predicted, and kernel method in the svm model
svm_model <- svm(dtm_matrix, classvec, kernel="linear")
summary(svm_model)
#inspect results
pred <- predict(svm_model, dtm)
table(pred,classvec)

In [ ]:
str(dtm$dimnames$Docs)


In [ ]:
#dtm$dimnames$Docs <- classvec

In [ ]:
#dtm$dimnames$Docs

In [ ]:
#str(dtm$dimnames$Docs)

In [ ]:
str(dtm)

In [ ]:
#str(dtm$dimnames$Docs[1000:1015])

In [ ]:
#str(dtm$dimnames$Docs[10:15])

In [ ]:
#str(dtm$dimnames$Docs[1570:1575])

In [ ]:
mat4 <- weightTfIdf(dtm)
mat4 <- as.matrix(mat4)

rm(dtm)

mem_used()



In [ ]:


set.seed(123)
# Compute and plot wss for k = 1 to k = 8
k.max <- 8
# Maximal number of clusters
data <- mat4
wss <- sapply(1:k.max, function(k){kmeans(mat4, k, nstart=10 )$tot.withinss})
plot(1:k.max, wss, type="b", pch = 19, frame = FALSE, xlab="Number of clusters K", ylab="Total within-clusters sum of squares")
abline(v = 3, lty =2)


